<a href="https://colab.research.google.com/github/Tony-TTran/Tony-TTran/blob/main/Customer_Churn_Predictor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
#!/usr/bin/env python
# coding: utf-8



import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from google.colab import drive
drive.mount('/content/drive')
data = pd.read_csv('/content/drive/My Drive/Colab Notebooks/Data Sets/Customer-Churn.csv', on_bad_lines='skip', engine="python")
data.info()

#Chaning our label to numerical value
churn = pd.Series(np.where(data.Churn == 'Yes', 1, 0),
          data.index)

data['Churn'] = churn



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   customerID        7043 non-null   object 
 1   gender            7043 non-null   object 
 2   SeniorCitizen     7043 non-null   int64  
 3   Partner           7043 non-null   object 
 4   Dependents        7043 non-null   object 
 5   tenure            7043 non-null   int64  
 6   PhoneService      7043 non-null   object 
 7   MultipleLines     7043 non-null   object 
 8   InternetService   7043 non-null   object 
 9   OnlineSecurity    7043 non-null   object 
 10  OnlineBackup      7043 non-null   object 
 11  DeviceProtection  7043 non-null   object 
 12  TechSupport       7043 non-null   object 
 13  StreamingTV       7043 non-null   object 
 14  Streami

In [4]:
#Splitting data into training and testing data
#Label (y) = Churn
#Feature (x) = everything else
#80% data will be used for training, 20% used for testing

from sklearn.model_selection import train_test_split
x = data.drop('Churn',axis=1)
y = data['Churn']
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size = 0.2)

#Rejoining the training data into one data set
training_data = x_train.join(y_train)


#data consist of string categorical variables that need to be change into numerical variables
training_data


,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
1771,7156-MHUGY,Male,1,No,No,13,Yes,No,Fiber optic,No,...,No,No,Yes,Yes,Month-to-month,Yes,Electronic check,90.50,1201.15,1
4948,3446-QDSZF,Female,0,No,No,4,Yes,No,DSL,No,...,No,No,Yes,No,Month-to-month,No,Credit card (automatic),55.50,227.35,0
1790,3096-IZETN,Female,0,No,No,12,Yes,No,Fiber optic,No,...,No,No,No,Yes,Month-to-month,No,Bank transfer (automatic),78.10,947.3,1
6500,9236-NDUCW,Female,0,No,No,21,No,No phone service,DSL,Yes,...,No,Yes,No,No,Two year,No,Mailed check,35.10,770.4,0
4512,0707-HOVVN,Female,1,No,No,70,Yes,Yes,DSL,No,...,Yes,Yes,Yes,No,Two year,Yes,Bank transfer (automatic),75.50,5212.65,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2727,3387-VATUS,Male,0,No,No,5,Yes,Yes,Fiber optic,No,...,No,No,Yes,Yes,Month-to-month,Yes,Bank transfer (automatic),94.85,462.8,1
1231,0384-RVBPI,Male,0,No,No,20,Yes,No,DSL,Yes,...,Yes,No,No,Yes,Month-to-month,No,Credit card (automatic),64.40,1398.6,0
1059,0376-YMCJC,Male,0,No,No,23,Yes,Yes,Fiber optic,No,...,No,Yes,Yes,No,Month-to-month,Yes,Electronic check,90.60,1943.2,1
609,3419-SNJJD,Female,1,Yes,No,65,Yes,Yes,Fiber optic,Yes,...,Yes,No,Yes,Yes,Month-to-month,No,Bank transfer (automatic),105.25,6786.4,1


In [5]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

#Drop Customer Id as it is unique to ea customer and does not play a role in whether the customer has churn
x = data.drop('customerID',axis=1)

catergorical_features = ['gender','Partner','Dependents','PhoneService'
                        ,'MultipleLines','InternetService','OnlineSecurity','OnlineBackup',
                        'DeviceProtection','TechSupport','StreamingTV','StreamingMovies',
                        'Contract','PaperlessBilling','PaymentMethod','TotalCharges']
one_hot = OneHotEncoder()
transformer = ColumnTransformer([('one_hot',
                                  one_hot,
                                  catergorical_features)],remainder = 'passthrough')

transform_x = transformer.fit_transform(x)
transform_x



<7043x6576 sparse matrix of type '<class 'numpy.float64'>'
	with 129774 stored elements in Compressed Sparse Row format>

In [6]:

pd.DataFrame(transform_x.toarray())

from sklearn.linear_model import LinearRegression

x_train,x_test,y_train,y_test = train_test_split(transform_x,y,test_size = 0.2)


reg = LinearRegression().fit(x_train, y_train)

reg.score(x_test,y_test)


0.9999992951346528

In [7]:

from sklearn.ensemble import RandomForestRegressor

forest = RandomForestRegressor()
forest.fit(x_train, y_train)



RandomForestRegressor()

In [8]:
forest.score(x_test,y_test)

1.0